# Registration Notebook

In [ ]:
from organoid import preprocessing
from organoid import reconstruction
import numpy as np
import tifffile
from pathlib import Path

Before the registration, if necesary, plot the positions of all samples, to match the two sides

In [ ]:
path = ...
reconstruction.plot_positions(
    path_bottom_positions = Path(path) / f"multipoints_bottom.xml",
    path_top_positions = Path(path) / f"multipoints_top.xml",
)

Give the path to your data

In [ ]:
path = ...
path_to_bin = ...
channels = ["dapi","ecad"]
list_ref = ["data_bot"]
list_float = ["data_top"]

Create folder structure

In [ ]:
reconstruction.create_folders(
    path, list_ref=list_ref, list_float=list_float, channels=channels
)

Register automatically

In [ ]:
# from now on, we consider only one sample. If you have multiple samples, you can loop : list_ref[i]
filename_ref = list_ref[0]
filename_float = list_float[0]

channel_reference = "dapi"

# register the reference channel first
reconstruction.register(
    path_data = Path(path) / f"{filename_ref}/raw",
    path_transformation = Path(path) / f"{filename_ref}/trsf",
    path_registered_data = Path(path) / f"{filename_ref}/registered",
    path_to_bin = path_to_bin,
    reference_image = f"{filename_ref}_{channel_reference}.tif",
    floating_image = f"{filename_float}_{channel_reference}.tif",
    input_voxel = [1,1,1],
    output_voxel = [1,1,1],
    compute_trsf = 1,
    init_trsfs = [
        ["rot", "Z", 180, "rot", "Y", 180, "trans", "Z", -40]
    ],  # example of transformation if the sample has been flipped between the 2 views
    test_init = 0,
    trsf_type = "rigid",
    depth = 3,
    bbox = 1,
    save_json = Path(path) / f"{filename_ref}",
)
# and the others using the same transformation
for channel in channels[
    1:
]:  # (we skip the reference channel that we assumed is the first)
    print(channel)
    reconstruction.register(
        path_data = Path(path) / f"{filename_ref}/raw",
        path_transformation = Path(path) / f"{filename_ref}/trsf",
        path_registered_data = Path(path) / f"{filename_ref}/registered",
        path_to_bin = path_to_bin,
        reference_image = f"{filename_ref}_{channel_reference}.tif",
        floating_image = f"{filename_float}_{channel_reference}.tif",
        input_voxel = [1,1,1],
        output_voxel = [1, 1, 1],
        compute_trsf = 0 # we do not compute the transformation, we use the same transformation as the reference channel
    )

Optional : Registration using landmarks (if the automatic registration fails)

In [ ]:
# optional : if the automatic registration is not satisfying, you can give some initial transformations to the function
# once you have annotated landmarks in the reference and floating images, give these landmarks to the manual_registration fct
filename_ref = list_ref[0]
filename_float = list_float[0]
scale = (1, 1, 1)
path_to_landmarks = ...
reference_landmarks = tifffile.imread(
     Path(path_to_landmarks) / f"reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"floating_landmarks.tif"
)
reference_landmarks = reference_landmarks.astype(np.uint16)
floating_landmarks = floating_landmarks.astype(np.uint16)
channel = "dapi"
rot, trans1, trans2 = reconstruction.manual_registration_fct(
    reference_landmarks = reference_landmarks,
    floating_landmarks = floating_landmarks,
    scale=(1, 1, 1),
)

init_trsfs = [
    [
        "trans",
        "X",
        -trans1[2],
        "trans",
        "Y",
        -trans1[1],
        "trans",
        "Z",
        -trans1[0],
        "rot",
        "X",
        -rot[0],
        "rot",
        "Y",
        -rot[1],
        "rot",
        "Z",
        -rot[2],
        "trans",
        "X",
        -trans2[2],
        "trans",
        "Y",
        -trans2[1],
        "trans",
        "Z",
        -trans2[0],
    ]
]
reconstruction.register(
    path_data = Path(path_to_landmarks) / f"{filename_ref}/raw",
    path_transformation = Path(path_to_landmarks) / f"{filename_ref}/trsf",
    path_registered_data = Path(path_to_landmarks) / f"{filename_ref}/registered",
    path_to_bin = path_to_bin,
    reference_image = f"{filename_ref}_{channel}.tif",
    floating_image = f"{filename_float}_{channel}.tif",
    input_voxel = [1,1,1],  # if you use landmarks, better use the same voxel size in and out
    output_voxel = [1,1,1],
    compute_trsf = 1,
    init_trsfs = init_trsfs,
    test_init = 0,
    trsf_type = "rigid",
    depth = 3,
    bbox = 0,
    save_json = "",
)

Napari visualization

In [ ]:
filename_ref = list_ref[0]
filename_float = list_float[0]
channel = channels[0]
scale = (1, 1, 1)
reconstruction.check_napari(
    path_data = Path(path) / f"{filename_ref}",
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    scale=scale,
)

Fuse the 2 registered sides into one array

In [ ]:
filename_ref = list_ref[0]
filename_float = list_float[0]
for ch in channels:
    reconstruction.fuse_sides(
        path_registered_data = Path(path) / f"{filename_ref}/registered",
        reference_image_reg = f"{filename_ref}_{ch}.tif",
        floating_image_reg = f"{filename_float}_{ch}.tif",
        folder_output = Path(path) / f"{filename_ref}/fused",
        name_output = rf"data_{ch}_fused.tif",
        slope_coeff = 10,
    )

Merge all the channels in one hyperstack image

In [ ]:
# the images should be named 'sampleid_channel_fused.tif', eg '1_dapi_fused.tif'
reconstruction.write_hyperstacks(
    Path(path) / f"{filename_ref}/fused",
    sample_id = "data",
    channels= channels
)

Debug : register the landmarks images and plot the result to see if the transformation is correct

In [ ]:
path_to_landmarks = ...

init_trsfs=[["rot", "Z", 180]]
reconstruction.register(
    path_data = Path(path_to_landmarks) / f"{filename_ref}/raw",
    path_transformation = Path(path_to_landmarks) / f"{filename_ref}/trsf",
    path_registered_data = Path(path_to_landmarks) / f"{filename_ref}/registered",
    path_to_bin=path_to_bin,
    reference_image=f"reference_landmarks.tif",  # they have to be in 16bit to be registered
    floating_image=f"floating_landmarks.tif",
    input_voxel=[1, 1, 1],
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    init_trsfs=init_trsfs,
    test_init=1,  # keep test_init=1 to see only the provided tranformation
    trsf_type="rigid",
    depth=3,
    save_json = Path(path) / f"{filename_ref}",
)

# loading the input to add articially the center of mass of the points, to see how well they align

reference_landmarks = tifffile.imread(
     Path(path_to_landmarks) / f"registered/reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"registered/floating_landmarks.tif"
)
reference_landmarks = reconstruction.add_centermass(reference_landmarks)
floating_landmarks = reconstruction.add_centermass(floating_landmarks)

# loading the raw landmarks, before the registartion
reference_landmarks_notreg = tifffile.imread(
     Path(path_to_landmarks) / f"raw/reference_landmarks.tif"
)
floating_landmarks_notreg = tifffile.imread(
    Path(path_to_landmarks) / f"raw/floating_landmarks.tif"
)
reference_landmarks_notreg = reconstruction.add_centermass(reference_landmarks_notreg)
floating_landmarks_notreg = reconstruction.add_centermass(floating_landmarks_notreg)

# plotting the center of the image
image_center = np.zeros_like(reference_landmarks)
z, y, x = np.array(reference_landmarks.shape) / 2
image_center[
    int(z) - 10 : int(z) + 10,
    int(y) - 10 : int(y) + 10,
    int(x) - 10 : int(x) + 10,
] = 100

reconstruction.check_napari(
    path_data=rf"{path}\{filename_ref}",
    reference_image=reference_landmarks,
    floating_image=floating_landmarks,
    additional_images=[reference_landmarks_notreg, floating_landmarks_notreg, image_center],
    names_additional_images=[
        "reference_landmarks_notreg",
        "floating_landmarks_notreg",
        "image_center",
    ],
    scale=scale,
    labels=True
)